For this notebook you need:
- Graph csv and json files
- Table with an orthogroups column and a column for every species containing which genes correspond to that orthogroup
- results from `find_enrichments.py` (see preprocessing)

# Reading the graph
The read_graph function uses the json and csv files corresponding to the graph as well as the table which maps orthogroups to genes to create a networkx graph.
- Node name is the name in the 'Node ID' column in the csv file
- Each node has the following attributes:
    - orthogroups: List of orthogroups
    - \<species label\>: Genes in that species corresponding to that orthogroup

In [1]:
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Reads a graph from a json file and returns it as a networkx graph object
def read_graph(path_to_json, path_to_csv, path_to_orthogroups):
    """
    Construct a graph with json with the graph structure, csv with the graph node info, and a tsv orthogroup
    
    input:
        path_to_json:   json graph file
        path_to_csv:    csv file with the orthogroups
        orthogroup_csv: csv file with orthogroup and corresponding lists of genes
    output:
        networkx graph where each node has the following attributes:
        - orthogroups containing a list of every orthogroup
        - <species name> for every species column in the orthogroup_csv file 
          containing a list of genes contained by the orthogroups
    
    Note: In the future I should modify this to take dataframes instead of paths since this requires an exact format of the data,
    whereas its much easier to just reformat a dataframe.
    """
    # Read the graph json file
    with open(path_to_json, 'r') as fi:
        graph_data = json.load(fi)
    
    # Read the graph csv
    node_info_df = pd.read_csv(path_to_csv)
    node_info_df = node_info_df.set_index(['Ortho Index', 'Node ID'])
    node_info_df = node_info_df.drop('Unnamed: 0', axis=1)
    
    # Read the orthogroup tsv
    orthogroup_df = pd.read_csv(path_to_orthogroups, delimiter='\t')
    orthogroup_df = orthogroup_df.set_index('Orthogroup')
    
    # Construct the graph with our data
    G = nx.graph.Graph()
    for node, og_indices in graph_data['nodes'].items():
        # Find which orthogroups are in the node
        orthogroups_in_node = [node_info_df.loc[og_idx, node]['Orthogroup'] for og_idx in og_indices]
        
        # Add that node and give it an attribute corresponding to its orthogroups
        G.add_node(node, orthogroups=orthogroups_in_node)
        
        # For every species, add the genes belonging to the orthogroups to the node
        for species in orthogroup_df.columns:
            genes = []
            for orthogroup in orthogroups_in_node:
                value = orthogroup_df.loc[orthogroup][species]
                if not type(value) == float:
                    # Sometimes a species does not have a gene in the orthogroup, in that case it is 'nan', which is type float
                    genes.extend(value.split(', '))
            G.nodes.get(node)[species] = genes # Assign the attributes   
    
    # Add the edges to the graph
    G.add_edges_from(graph_data['edges'])
    
    return G

def add_enrichments(graph, enrichment_prefix, enrichment_suffix):
    """
    graph: networkx object to add these attributes to
    result_prefix & result suffix: text that goes before and after the node name in the file
    """
    for node_name, node_attributes in graph.nodes.items():
        # This assumes that the file has the node name in the name
        node_file_path = Path(f'{enrichment_prefix}{node_name}{enrichment_suffix}')
        if node_file_path.exists():
            node_data = pd.read_csv(node_file_path, delimiter='\t')
            enrichment_mask = node_data['enrichment'] == 'e'
            purified_mask = node_data['enrichment'] == 'p'
            node_attributes['enriched'] = list(node_data[enrichment_mask]['name'])
            node_attributes['purified'] = list(node_data[purified_mask]['name'])
        else:
            node_attributes['enriched'] = []
            node_attributes['purified'] = []


Now, let's read a graph and look at/interact with it

In [2]:
orthogroup_path = "data/Orthogroups_filtered.tsv"
FilterBy_family = read_graph("data/Mapper_Graphs/FilterBy_family_Cubes_70_Overlap_80.json", 
                            "data/Mapper_Graphs/FilterBy_family_Cubes_70_Overlap_80.csv", 
                            orthogroup_path)

In [3]:
add_enrichments(FilterBy_family, "data/find_enrichment_results/FilterBy_family_Cubes_70_Overlap_80/study_", ".txt_enrichment.tsv")

To visualize, we will use the [pyvis](https://pyvis.readthedocs.io/en/latest/index.html) package

In [4]:
from pyvis import network as net

g = net.Network(notebook=True, width='800px', height='600px')
g.from_nx(FilterBy_family)
g.show("example.html")

If we set the `title` attribute to a node in this graph, it shows a text box when you hover over it.

In [5]:
def annotate_with_enrichment(graph):
    for node_attributes in graph.nodes.values():
        enrichment = "<br>".join(node_attributes['enriched'])
        if enrichment:
            node_attributes['title'] = enrichment
        else:
            node_attributes['title'] = "No enrichment found"

In [6]:
annotate_with_enrichment(FilterBy_family)

Now if we hover over the nodes, it will show a list of nodes which are enriched.

In [7]:
g = net.Network(notebook=True, width='800px', height='600px')
g.from_nx(FilterBy_family)
g.show("example.html")

## Coloring by GO terms

We can write code to recolor nodes in the graph on certain conditions, such as if a node contains enrichment in some GO terms

In [8]:
def reset_color(graph, color=None):
    # For setting all the nodes in the graph to the same color
    for node in graph.nodes.values():
        node['color'] = color

def color_if_enriched(graph, go_term, color='#7acc7d'):
    # Only recolors a node if it matches exactly
    for node in graph.nodes.values():
        if go_term in node['enriched']:
            node['color'] = color
            
def color_if_contains(graph, keywords, color='#7acc7d'):
    for node in graph.nodes.values():
        for go_term in node['enriched']:
            if sum([keyword in go_term for keyword in keywords]):
                node['color'] = color
                break

def color_no_enrichment(graph, color='#aaaaaa'):
    for node in graph.nodes.values():
        if not node['enriched']:
            node['color'] = color

In [9]:
reset_color(FilterBy_family)
color_if_enriched(FilterBy_family, 'chloroplast')
g = net.Network(notebook=True, width='800px', height='600px')
g.from_nx(FilterBy_family)
g.show("example.html")

In [10]:
reset_color(FilterBy_family)
color_no_enrichment(FilterBy_family)
color_if_contains(FilterBy_family, ['light'], color='#b7668d')
g = net.Network(notebook=True, width='800px', height='600px')
g.from_nx(FilterBy_family)
g.show("example.html")

If we want to look at what enriched GO terms we have in our graph, we can do the following

In [11]:
all_enriched_GO_terms = set()
for node in FilterBy_family.nodes.values():
    all_enriched_GO_terms.update(set(node['enriched']))
print(f"# of unqiue enriched GO terms: {len(all_enriched_GO_terms)}")
all_enriched_GO_terms

# of unqiue enriched GO terms: 164


{'2-oxo-4-hydroxy-4-carboxy-5-ureidoimidazoline decarboxylase activity',
 'ATP binding',
 'Golgi apparatus',
 'Golgi apparatus subcompartment',
 'Golgi cis cisterna',
 'Golgi cisterna',
 'Golgi trans cisterna',
 'adenyl nucleotide binding',
 'adenyl ribonucleotide binding',
 'anchored component of membrane',
 'anchored component of plasma membrane',
 'anchoring junction',
 'anion binding',
 'apoplast',
 'binding',
 'biological process involved in interspecies interaction between organisms',
 'biological regulation',
 'biological_process',
 'biosynthetic process',
 'bounding membrane of organelle',
 'carbohydrate derivative binding',
 'carbon-carbon lyase activity',
 'carboxy-lyase activity',
 'catalytic activity',
 'cation binding',
 'cell junction',
 'cell wall',
 'cell-cell junction',
 'cellular anatomical entity',
 'cellular biosynthetic process',
 'cellular component organization',
 'cellular component organization or biogenesis',
 'cellular lipid metabolic process',
 'cellular nit

In [12]:
FilterBy_stress = read_graph("data/Mapper_Graphs/FilterBy_stress_Cubes_90_Overlap_80.json", 
                             "data/Mapper_Graphs/FilterBy_stress_Cubes_90_Overlap_80.csv", 
                             orthogroup_path)

In [13]:
add_enrichments(FilterBy_stress, "data/find_enrichment_results/FilterBy_stress_Cubes_90_Overlap_80/study_", ".txt_enrichment.tsv")
annotate_with_enrichment(FilterBy_stress)

In [14]:
reset_color(FilterBy_stress)
color_if_contains(FilterBy_stress, ['response to stress'], color='#e25738')

In [16]:
g = net.Network(notebook=True, height='1000px', width='1500px')
g.from_nx(FilterBy_stress)
g.show("example.html")

In [17]:
FilterBy_tissue = read_graph("data/Mapper_Graphs/FilterBy_tissue_Cubes_80_Overlap_60.json", 
                            "data/Mapper_Graphs/FilterBy_tissue_Cubes_80_Overlap_60.csv", 
                            orthogroup_path)

In [18]:
g = net.Network(notebook=True, height='1000px', width='1500px')
g.from_nx(FilterBy_tissue)
g.show("example.html")